In [2]:
# determine type and occupation of ACMI creators, for Wikidata upload.

# # preprocess data sources.

# parse acmi api to pull dataframe.

import json
import numpy
import pandas
import pathlib
import pydash
import requests
import tqdm

acmi_path = pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'json' / 'works'
acmi_files = [filename for filename in acmi_path.iterdir() if filename.suffix == '.json' and 'index' not in filename.name]

acmi_dataframe = pandas.DataFrame(columns=['acmi_creator_id', 'acmi_creator_name', 'acmi_creator_role'])
    
for acmi_file in tqdm.tqdm(acmi_files):

    with open(acmi_file, encoding='utf-8') as acmi_data:
        acmi_data = json.load(acmi_data)

    if 'creators_primary' in acmi_data:
        for y in acmi_data['creators_primary']:
            acmi_dataframe.loc[len(acmi_dataframe)] = [y['creator_id'], y['name'], y['role']]

    if 'creators_other' in acmi_data:
        for y in acmi_data['creators_other']:
            acmi_dataframe.loc[len(acmi_dataframe)] = [y['creator_id'], y['name'], y['role']]

print(len(acmi_dataframe))
acmi_dataframe.head()

/Users/paulduchesne/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
100%|██████████| 42921/42921 [28:06<00:00, 25.44it/s] 

119891


,acmi_creator_id,acmi_creator_name,acmi_creator_role
0,83676,Jessica Hobbs,director
1,33376,Jacquelin Perske,writer
2,76678,Claudia Karvan,producer
3,13073,John Edwards,producer
4,7906,New South Wales Film and Television Office,production company


In [1]:
dataframe = acmi_dataframe.copy()

acmi_creator = pandas.read_csv(pathlib.Path.cwd().parents[0] / 'acmi-api' / 'app' / 'tsv' / 'creators.tsv', delimiter='\t')
acmi_creator = acmi_creator.loc[acmi_creator.source.isin(['Vernon'])]
acmi_creator = acmi_creator[['id', 'name', 'source', 'source_identifier']]

acmi_creator = acmi_creator.rename(columns={'id':'acmi_creator_id'})

dataframe = pandas.merge(dataframe, acmi_creator, on='acmi_creator_id', how='left')

vernon_data = pandas.concat([pandas.read_csv(pathlib.Path.home() / 'Desktop' / 'vernon_manufacture.csv'), 
                             pandas.read_csv(pathlib.Path.home() / 'Desktop' / 'vernon_publisher.csv')])

vernon_data = vernon_data[['System ID', 'Record Type',  'Person/Corporate Type', 'Lastname']]
vernon_data = vernon_data.rename(columns={'System ID':'source_identifier'})
dataframe = pandas.merge(dataframe, vernon_data, on='source_identifier', how='left')

dataframe.loc[dataframe.acmi_creator_role.isin(['production company']), 'wikidata_type'] = 'Q11396960'
dataframe.loc[dataframe.acmi_creator_role.isin(['director', 'producer','co-director', 
        'co-producer', 'cast', 'editor', 'cinematographer', 'writer'
        'creator', 'producer/director', 'co-producer', 'music', 'sound'
        
        ]), 'wikidata_type'] = 'Q5'
dataframe.loc[dataframe['Record Type'].isin(['Person']), 'wikidata_type'] = 'Q5'





dataframe.to_csv(pathlib.Path.cwd().parents[0] / 'data' / 'creator_roles' / 'creator_roles.csv', index=False)


print(len(dataframe))
dataframe.head()

NameError: name 'acmi_dataframe' is not defined

In [4]:
# first up, attribute type; then occupation
test = pandas.DataFrame(dataframe.acmi_creator_role.value_counts())

print(len(test))
test.head(20)
#print(len(dataframe.acmi_creator_role.unique()))

189


,count
acmi_creator_role,
production company,43359
director,18100
producer,15703
distributor,10427
co-producer,8927
creator,4117
producer/director,4103
co-director,2932
cast,1671
